# Download crystal structures from PDB

Download PDB structures of a target protein, saving them as `pdb` files.

In [10]:
import pandas as pd
import numpy as np
import nglview as nv
from pathlib import Path
import sys
sys.path.insert(0, '../..')
from helper_modules.get_pdb_ids_from_uniport import *

## Inputs
This notebook requires to specify the following values
- `prot_name`: the name of the protein.
- `uniprot_id`: Uniprot Accession number.
- `ref_struc_id`: PDB id of the reference structure.
- `ref_ligand_name`: Three letters name of the cocrystalized ligand in the reference structure.


In [11]:
prot_name       = 'hsp90'
uniprot_id      = 'P07900'
ref_struc_id    = '1uyg'
ref_ligand_name = 'PU2'

In [12]:
seq_prot = get_seq_from_uniprot(uniprot_id)
print(seq_prot)
print(f'\nThere are {len(seq_prot)} residues.')

MPEETQTQDQPMEEEEVETFAFQAEIAQLMSLIINTFYSNKEIFLRELISNSSDALDKIRYESLTDPSKLDSGKELHINLIPNKQDRTLTIVDTGIGMTKADLINNLGTIAKSGTKAFMEALQAGADISMIGQFGVGFYSAYLVAEKVTVITKHNDDEQYAWESSAGGSFTVRTDTGEPMGRGTKVILHLKEDQTEYLEERRIKEIVKKHSQFIGYPITLFVEKERDKEVSDDEAEEKEDKEEEKEKEEKESEDKPEIEDVGSDEEEEKKDGDKKKKKKIKEKYIDQEELNKTKPIWTRNPDDITNEEYGEFYKSLTNDWEDHLAVKHFSVEGQLEFRALLFVPRRAPFDLFENRKKKNNIKLYVRRVFIMDNCEELIPEYLNFIRGVVDSEDLPLNISREMLQQSKILKVIRKNLVKKCLELFTELAEDKENYKKFYEQFSKNIKLGIHEDSQNRKKLSELLRYYTSASGDEMVSLKDYCTRMKENQKHIYYITGETKDQVANSAFVERLRKHGLEVIYMIEPIDEYCVQQLKEFEGKTLVSVTKEGLELPEDEEEKKKQEEKKTKFENLCKIMKDILEKKVEKVVVSNRLVTSPCCIVTSTYGWTANMERIMKAQALRDNSTMGYMAAKKHLEINPDHSIIETLRQKAEADKNDKSVKDLVILLYETALLSSGFSLEDPQTHANRIYRMIKLGLGIDEDDPTADDTSAAVTEEMPPLEGDDDTSRMEEVD

There are 732 residues.


- Get from the Uniprot database all the associated PDB structures to the given Uniprot id.

In [13]:
# Get the pdbids and some metadata from uniprot
df_pdb_ids = pdb_ids_from_uniprot(uniprot_id)
df_pdb_ids.head()

,pdb_id,method,resolution,chain,start,end,seq_len
0,1byq,X-ray,1.50,A,9,236,228
1,1osf,X-ray,1.75,A,9,223,215
2,1uy6,X-ray,1.90,A,2,236,235
3,1uy7,X-ray,1.90,A,2,236,235
4,1uy8,X-ray,1.98,A,2,236,235


In [14]:
df_pdb_ids.shape

(309, 7)

## Get structural information from the reference structure

### For HSP90 only

<mark>NOTE:</mark> As show in the `df_pdb_ids` table, the structures vary in their sequence coverage. The following cells aim to preserve only those structures containing the binding site of the protein by using the reference structure, which PDB id was provided above.

In [15]:
# Get the reference structure and its sequence and residue positions
seq_cry, positions_cry = get_structure_sequence(ref_struc_id)
print(len(seq_cry))
print(positions_cry[0], positions_cry[-1])
seq_cry

208
16 223


'EVETFAFQAEIAQLMSLIINTFYSNKEIFLRELISNSSDALDKIRYESLTDPSKLDSGKELHINLIPNKQDRTLTIVDTGIGMTKADLINNLGTIAKSGTKAFMEALQAGADISMIGQFGVGFYSAYLVAEKVTVITKHNDDEQYAWESSAGGSFTVRTDTGEPMGRGTKVILHLKEDQTEYLEERRIKEIVKKHSQFIGYPITLFVE'

In [16]:
# Get the pdb id of those structures matching the positions of the 
# reference structure
df_sel_pdbids = get_useful_pdbids(df_pdb_ids, 
                                  positions_cry,
                                  thr_tol = (3,3))
df_sel_pdbids.tail(10)

,pdb_id,method,resolution,chain,start,end,seq_len
299,6n8x,X-ray,1.49,A,1,236,236
300,6olx,X-ray,1.44,A,1,236,236
301,6tn4,X-ray,1.27,A,9,236,228
302,6tn5,X-ray,1.17,A,9,236,228
303,6u98,X-ray,1.50,A,2,236,235
304,6u99,X-ray,1.60,A,2,236,235
305,6u9a,X-ray,1.65,A,2,236,235
306,6u9b,X-ray,1.75,A,2,236,235
307,7lsz,X-ray,1.70,A,1,293,293
308,7lt0,X-ray,1.70,A,1,293,293


In [17]:
ref_prot = parsePDB(ref_struc_id).select('chain A')
pocket_sel = ref_prot.select(
    f'within 7 of resname {ref_ligand_name} and protein')
pocket_residues = [str(i) 
                   for i in np.unique(
                       pocket_sel.getResnums())
                  ]
pocket_residues_str = ' '.join(pocket_residues)
lig_resnum = ref_prot.select(
                       f'resname {ref_ligand_name}'
                 ).getResnums()[0]

- Visualize using `nglview`

In [18]:
view = nv.show_prody(ref_prot)
view.clear_representations()
view.add_representation('cartoon', 
    selection='protein and chain A', color='orange')
view.add_cartoon(
    selection = pocket_residues_str, 
    color='cyan')
view.add_licorice(
    selection = pocket_residues_str, 
    color='cyan')
view.add_representation(
    'spacefill', selection = f'{lig_resnum}')
view.parameters = {
    "fogNear": 0, "fogFar": 100,
    "backgroundColor": "black",
}
view

NGLWidget()

In [19]:
view.render_image()

Image(value=b'', width='99%')

In [20]:
view._display_image()

## Download the PDB structures
1. Create the output directories

In [21]:
# Define the output directories
OUT_MAIN       = './pdb_structures'
OUT_DIR_RAW    = f'./{OUT_MAIN}/raw_structures'
OUT_DIR_CHAINS = f'./{OUT_MAIN}/pdb_chains'
OUT_DIR_LIGS   = f'./{OUT_MAIN}/raw_ligands'

for path in [OUT_DIR_RAW, OUT_DIR_CHAINS, OUT_DIR_LIGS]:
    Path(path).mkdir(parents = True, exist_ok = True)

2. Use `prody` to download and process the pdb chain belonging to the target protein.

In [22]:
from prody import parsePDB, matchAlign, writePDB

# We will use the reference structure to define the start and end
ref_chain = ref_prot.select('protein and chain A')
# From the reference structure
start_res = positions_cry[0]
end_res = positions_cry[-1]

# Overlap and Sequence Identity thresholds
overlap = 80
seqid   = 95
residues_algn = pocket_residues_str 

# Iterate over the list of pdb_ids and download the structures
for i, (pdb_id, chid) in df_sel_pdbids[['pdb_id', 'chain']].iterrows():
    print(pdb_id)
    # Download the full structure
    raw_pdb = parsePDB(pdb_id, folder = OUT_DIR_RAW, model = 1)
    # Keep the protein chain
    pdb_chain = raw_pdb.select(f'protein and chain {chid}')
    if pdb_chain is None:
        # If the reported chid is wrong, try with the
        # first id of the retrieved structure
        # this was add for the HSP90 structure and the 6tn4 and 6tn5 pdbids
        chid = np.unique(raw_pdb.getChids())[0] 
        pdb_chain = raw_pdb.select(f'protein and chain {chid}')
    pdb_chain.setChids("A")
    # If the structure has more 
    min_start_res = pdb_chain.getResnums()[0]
    # If the pdb includes extra residues at the N-terminal
    min_start_res = 0 if min_start_res < 0 else min_start_res
    _start_res    = min(min_start_res, start_res)
    # Keep the same starting and ending points as the reference
    pdb_chain = pdb_chain.select(f'resnum {_start_res}to{end_res}')
    # Make the structure alignment regarding the ref_prot
    pdb_alg = matchAlign(
                mobile = pdb_chain,
                target = ref_chain,
                # If the overlap or the seqid thresholds are not met
                # the alignment will return None
                overlap = overlap,
                tarsel  = f'chain A ca and resnum {residues_algn}'
              )
    # If the alignment retuned None, skip the next steps
    if not pdb_alg:
        continue
    # Get the aligned structure
    protein = pdb_alg[0]
    # Set the same chain id (A) for all structures
    protein.setChids("A")
    # Save the PDB chain
    chain_pdb_file = f'{OUT_DIR_CHAINS}/{pdb_id}_A.pdb'
    writePDB(chain_pdb_file, protein)
    # Save the non proteic atoms (raw ligands)
    ligand = raw_pdb.select('hetatm')
    if ligand:
        raw_ligand_file = f'{OUT_DIR_LIGS}/{pdb_id}_lig.pdb'
        writePDB(raw_ligand_file, ligand)

1byq
1osf
1uy6
1uy7
1uy8
1uy9
1uyc
1uyd
1uye
1uyf
1uyg
1uyh
1uyi
1uyk
1uyl
1yc1
1yc3
1yc4
1yer
1yes
1yet
2bsm
2bt0
2byh
2byi
2bz5
2ccs
2cct
2ccu
2fwy
2fwz
2h55
2jjc
2k5b
2qf6
2qfo
2qg0
2qg2
2uwd
2vci
2vcj
2wi1
2wi2
2wi3
2wi4
2wi5
2wi6
2wi7
2xab
2xdk
2xdl
2xds
2xdu
2xdx
2xhr
2xht
2xhx
2xjg
2xjj
2xjx
2xk2
2ye2
2ye3
2ye4
2ye5
2ye6
2ye7
2ye8
2ye9
2yea
2yeb
2yec
2yed
2yee
2yef
2yeg
2yeh
2yei
2yej
2yi0
2yi5
2yi6
2yi7
2yjw
2yjx
2yk2
2yk9
2ykb
2ykc
2yke
2yki
2ykj
3b24
3b25
3b26
3b27
3b28
3bm9
3bmy
3d0b
3eko
3ekr
3ft5
3ft8
3hek
3hhu
3hyy
3hyz
3hz1
3hz5
3inw
3inx
3k97
3k98
3k99
3mnr
3o0i
3ow6
3owb
3owd
3qdd
3qtf
3r4m
3r4n
3r4o
3r4p
3r91
3r92
3rkz
3rlp
3rlq
3rlr
3t0h
3t0z
3t10
3t1k
3t2s
3tuh
3vha
3vhc
3vhd
3wha
3wq9
4awo
4awp
4awq
4b7p
4bqg
4bqj
4cwf
4cwn
4cwo
4cwp
4cwq
4cwr
4cws
4cwt
4eeh
4eft
4efu
4egh
4egi
4egk
4fcp
4fcq
4fcr
4hy6
4jql
4l8z
4l90
4l91
4l93
4l94
4lwe
4lwf
4lwg
4lwh
4lwi
4nh7
4nh8
4o04
4o05
4o07
4o09
4o0b
4r3m
4u93
4w7t
4xip
4xiq
4xir
4xit
4ykq
4ykr
4ykt
4yku
4ykw
4ykx
4yky
4ykz


Finished!